# Równania różniczkowe zwyczajne


<h3> Przydatne linki: </h3>

* Układ Lorenza: https://en.wikipedia.org/wiki/Lorenz_system
* Metoda Eulera: http://tutorial.math.lamar.edu/Classes/DE/EulersMethod.aspx
* Backward Euler: https://en.wikipedia.org/wiki/Backward_Euler_method
* Metoda Rungego-Kutty (4-th order): https://en.wikipedia.org/wiki/Runge–Kutta_methods#Common_fourth-order_Runge.E2.80.93Kutta_method 
* boost-odeint : https://www.boost.org/doc/libs/1_66_0/libs/numeric/odeint/doc/html/index.html
 
<h3> Krótki wstęp </h3>

Mamy następujący układ trzech ODE:

${\begin{cases}{\dot  x}=\sigma y-\sigma x\\{\dot  y}=-xz+rx-y\\{\dot  z}=xy-bz\end{cases}}$
 
Układ Lorenza (bo takowy widnieje powyżej) został przez niego stworzony z myślą modelowania konwekcji termicznej w atmosferze. To co jest ciekawe w tym układzie i co będzie Nas interesowało na tych zajęciach to fakt, że dla pewnego zbioru parametrów układ ten zachowuje się chaotycznie. Wykres zmiennych w przestrzeni fazowej przedstawia tzw. atraktor Lorenza (poniżej).

![image.png](https://upload.wikimedia.org/wikipedia/commons/2/25/LorenzAttractor.png)

Jak łatwo się domyślić - celem tego laboratorium będzie implementacja metod rozwiązywania ODE, które umożliwią wizualizacje powyższego atraktora. 

Opisy poszczególnych metod znajdują się w linkach podanych w pierwszej sekcji instrukcji oraz w wykładach (nie streszczałem Wam tego, gdyż w linkach podanych przeze mnie jest dość szczegółowo - bez zbędnych wywodów matematycznych etc.)

<h3> Zadania </h3>


**Zadanie 1**
Proszę zapoznać się z materiałami załączonymi do tego laboratorium (zwłaszcza z opisami metod!).

**Zadanie 2** 
Proszę wykonać implementacje następujących metod rozwiązywania równań różniczkowych:
- metoda Eulera,
- modyfikacja metody Eulera (ang. Backward Euler method),
- metoda Rungego-Kutty 1 rzędu (It's a trap!),
- metoda Rungego-Kutty 2 rzędu (ang. midpoint method),
- metoda Rungego-Kutty 4 rzędu,

Za ich pomocą rozwiązać opisany powyżej układ równań. Wynik wyliczeń kolejnych kroków proszę przedstawić na wykresie (powinno się otrzymać powyższy atraktor). Środowisko i język tworzenia wykresu jest dowolny.

**Zadanie 3** 
Proszę dokonać porównania **teoretycznego** wszystkich powyższych metod. 

**Zadanie 4** 
Dane jest równanie różniczkowe (zagadnienie początkowe):

$y^{'} - kmysin(mx) = k^{2}msin(mx)cos(mx),  \hspace{1cm} y(x_{0})=a$

$x_{0}, x{k}, m, k$ - parametry zadawane, $a$ - wyliczane z rozwiązania dokładnego (poniżej).
Znaleźć rozwiązanie tego zagadnienia y(x) w przedziale $[x_{0}, x_{k}]$ metodą Eulera oraz metodą Rungego-Kutty. Eksperyment przeprowadzić dla różnej ilości kroków.

Porównać otrzymane rozwiązanie z rozwiązaniem dokładnym: $y(x) = e^{-kcos(mx)} - kcos(mx) + 1$

Stosownie przedstaw wyniki wybranych rozwiązań.

**Zadanie 5** (dla chętnych, żeby poznać bibliotekę boost)
Proszę rozwiązać układ Lorenza korzystając z funkcjonalności biblioteki **boost** (oficjalna dokumentacja i opis metod - https://www.boost.org).


### zad2

In [ ]:
vector<double> Euler_method(vector<double> mesh, function<double(double, double)> ftu, double u0){
    int n = mesh.size();
    double step = mesh[1] - mesh[0];
    vector<double> result(n);
    result[0] = u0;
    for(int i = 1; i < n; i++){
        result[i] = result[i-1] + ftu(mesh[i-1], result[i-1])*step;
    }
    return result;
}

In [ ]:
vector<double> backward_Euler_method(vector<double> mesh, function<double(double, double)> ftu, double u0){
    int n = mesh.size();
    double step = mesh[1] - mesh[0];
    vector<double> result(n);
    result[0] = u0;
    for(int i = 1; i < n; i++){
        result[i] = result[i-1];
        result[i] = result[i-1] + ftu(mesh[i], result[i])*step;
    }
    return result;
}

#### metoda Rungego-Kuty 1-rzędu to metoda Eulera

In [ ]:
vector<double> Rung_Kuta_2(vector<double> mesh, function<double(double, double)> ftu, double u0){
    int n = mesh.size();
    double step = mesh[1] - mesh[0];
    vector<double> result(n);
    double c1 = 0;
    double c2 = 1;
    auto k1 = [ftu](double t, double u, double h){ return ftu(t, u);};
    auto k2 = [ftu](double t, double u, double h){ return ftu(t + 0.5*h, u + 0.5*h*ftu(t, u));};

    result[0] = u0;
    for(int i = 1; i < n; i++){
        result[i] = result[i-1] + step*(c1*k1(mesh[i-1], result[i-1], step) + c2*k2(mesh[i-1], result[i-1], step));
    }
    return result;
}

In [ ]:
vector<double> Rung_Kuta_4(vector<double> mesh, function<double(double, double)> ftu, double u0){
    int n = mesh.size();
    double step = mesh[1] - mesh[0];
    vector<double> result(n);
    double c1 = 1;
    double c2 = 2;
    double c3 = 2;
    double c4 = 1;
    auto k1 = [ftu](double t, double u, double h){ return ftu(t, u);};
    auto k2 = [ftu, k1](double t, double u, double h){ return ftu(t + 0.5*h, u + 0.5*h*k1(t, u, h));};
    auto k3 = [ftu, k2](double t, double u, double h){ return ftu(t + 0.5*h, u + 0.5*h*k2(t, u, h));};
    auto k4 = [ftu, k3](double t, double u, double h){ return ftu(t + h, u + h*k3(t, u, h));};

    result[0] = u0;
    for(int i = 1; i < n; i++){
        result[i] = result[i - 1] + step / 6.0 * (c1 * k1(mesh[i - 1], result[i - 1], step) + c2 * k2(mesh[i - 1], result[i - 1], step) + c3 * k3(mesh[i - 1], result[i - 1], step) + c4 * k4(mesh[i - 1], result[i - 1], step));
    }
    return result;
}

### zad.3

Metoda Eulera jest najprostszą z metod sprowadza się do skorzystania ze sposobu analitycznego rozwiązania równania i obliczenia całki metodą prostokątów. Metoda Rungego-Kuty bazuje na rozwinięciu funkcji w szereg Taylora i fakcie, że w rozwinięciu występuje pochodna funkcji wyjściowej, której wartość jest podana w równaniu. w metodzie Eulera dokładność obliczeń zależy od kroku liczenia całki. W metodzie Rungego-Kuty wpływ ma również ilość składników sumy z szeregu Taylora, które weźmiemy pod uwagę przy obliczeniach.

### zad.4

In [ ]:
void test(){
    double k = 1.0;
    double m = 1.0;
    double x0 = 0;
    double xk = 18.0;

    auto ftu = bind(ftukm, _1, _2, k ,m);
    auto exact_sol = bind(exact_ftukm, k, m, _1);

    double a = exact_sol(x0);


    vector<int> n = {10, 20, 40, 50, 100, 300, 500, 1000};
    for(int i = 0; i < n.size(); i++){
        vector<double> mesh = generate_mesh(x0, xk, n[i]);
        vector<double> resEuler = Euler_method(mesh, ftu, a);
        vector<double> resRG = Rung_Kuta_4(mesh, ftu, a);
        save_result(mesh, resEuler, "euler_" + to_string(n[i]) + ".txt");
        save_result(mesh, resRG, "rung_kuta_" + to_string(n[i]) + ".txt");
    }

    vector<double> mesh = generate_mesh(x0, xk, 200);
    vector<double> res_exact(mesh.size());
    for(int i = 0; i < mesh.size(); i++) res_exact[i] = exact_sol(mesh[i]);
    save_result(mesh, res_exact, "exact_values.txt");
    
}

#### Wyniki dla kolejnych ilości kroków:

![](img/1.png)  
![](img/2.png)  
![](img/3.png)  
![](img/4.png)  
![](img/5.png)  
![](img/6.png)  

#  
Widać, że metoda Rungego-Kuty już dla 100 kroków pokazuje całkiem dokładny wynik, natomiast metoda Eulera dopiero dla 1000 kroków zbliża się do poprawnego rozwiązania